# Alessandro Castelli Kaggle: https://www.kaggle.com/code/alessandromajumba/ex5-ml

# Kaggle Competition: https://www.kaggle.com/competitions/cifar-10

# WANDB: https://wandb.ai/ales-2000-09/CIFAR-10?workspace=user-ales-2000-09

In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


## Import everything needed

In [80]:
import glob
from PIL import Image
import matplotlib.pyplot as plt
import collections
import math
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

## Unzip datasets

In [81]:
#!pip install py7zr

# WARNING: It can take a lot of time to uncompress!

In [82]:
#!python -m py7zr x /kaggle/input/cifar-10/train.7z

In [83]:
#!python -m py7zr x /kaggle/input/cifar-10/test.7z

In [84]:
data_dir = '/kaggle/working/'

In [85]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Label")

import wandb
wandb.login(key=secret_value_0)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [86]:
def read_csv_labels(fname):
    """Read `fname` to return a filename to label dictionary."""
    with open(fname, 'r') as f:
        # Skip the file header line (column name)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name, label in tokens))

labels = read_csv_labels(os.path.join(data_dir, '/kaggle/input/cifar-10/trainLabels.csv'))
print(f'Number training examples: {len(labels)}')
print(f'Number classes: {len(set(labels.values()))}')

Number training examples: 50000
Number classes: 10


In [87]:
def copyfile(filename, target_dir):
    """Copy a file into a target directory."""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_train_valid(data_dir, labels, valid_ratio):
    """Split the validation set out of the original training set."""
    # The number of examples of the class that has the fewest examples in the
    # training dataset
    n = collections.Counter(labels.values()).most_common()[-1][1]
    # The number of examples per class for the validation set
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                     'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'train', label))
    return n_valid_per_label

def reorg_test(data_dir):
    """Organize the testing set for data loading during prediction."""
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(os.path.join(data_dir, 'test', test_file),
                 os.path.join(data_dir, 'train_valid_test', 'test',
                              'unknown'))
        
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = read_csv_labels('/kaggle/input/cifar-10/trainLabels.csv')
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [88]:
wandb.init(project='CIFAR-10', save_code=True)
batch_size = 64
wandb.log({'Batch_size': batch_size})
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)

wandb: WARNING No program path found when generating artifact job source for a non-colab notebook run. See https://docs.wandb.ai/guides/launch/create-job
wandb: WARNING Source type is set to 'artifact' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Batch_size,▁
Epoch,▁
LR,▁
Training Loss,▁
Batch_size,64
Epoch,1
LR,0.003
Training Loss,1.6023


In [89]:
transform_train = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])
])

transform_test = torchvision.transforms.Compose([
    
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

In [90]:
train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_train) for folder in ['train', 'train_valid']]

valid_ds, test_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_test) for folder in ['valid', 'test']]

train_iter, train_valid_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last=True)
    for dataset in (train_ds, train_valid_ds)]

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

In [91]:
# Load pre-trained ResNet-18 model
resnet18 = torchvision.models.resnet18(pretrained=True)

# Modify the fully connected layer (fc) for the CIFAR-10 dataset (10 classes)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)

# Initialize the weights of the fully connected layer using Xavier normal initialization
nn.init.xavier_normal_(resnet18.fc.weight)

# Initialize the bias of the fully connected layer to zeros
nn.init.constant_(resnet18.fc.bias, 0)

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

In [92]:
lr = 0.003
wandb.log({'LR': lr})

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.SGD(resnet18.parameters(), lr=lr, momentum=0.9)

# Training loop
num_epochs = 32
for epoch in range(num_epochs):
    resnet18.train()
    total_loss = 0.0  # Initialize total loss for the epoch
    for inputs, labels in train_iter:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    average_loss = total_loss / len(train_iter)  # Calculate the average loss for the epoch
    print(f'Finish Epoch: {epoch + 1}, Training Loss: {average_loss}')
    wandb.log({'Training Loss': average_loss})
    wandb.log({'Epoch': epoch + 1})

    # Evaluation on the validation set
    resnet18.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in valid_iter:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet18(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    average_loss2 = val_loss / len(valid_iter)  
    print(f'Finish Epoch: {epoch + 1}, Validation Loss: {average_loss2}')
    wandb.log({'Validation Loss': average_loss2})

    accuracy = correct / total
    print(f'Validation Accuracy: {accuracy}')
    wandb.log({'Accuracy on validation': accuracy})
    
wandb.finish()

Finish Epoch: 1, Training Loss: 1.5992823978236865
Finish Epoch: 1, Validation Loss: 1.1701220296896422
Validation Accuracy: 0.7017227564102564
Finish Epoch: 2, Training Loss: 1.3400502006495492
Finish Epoch: 2, Validation Loss: 1.0842253038516412
Validation Accuracy: 0.742988782051282
Finish Epoch: 3, Training Loss: 1.268226931139212
Finish Epoch: 3, Validation Loss: 1.0413500788884285
Validation Accuracy: 0.7716346153846154
Finish Epoch: 4, Training Loss: 1.2235089237795107
Finish Epoch: 4, Validation Loss: 0.9836106430261563
Validation Accuracy: 0.7926682692307693
Finish Epoch: 5, Training Loss: 1.20176811247089
Finish Epoch: 5, Validation Loss: 0.9814935525258383
Validation Accuracy: 0.7930689102564102
Finish Epoch: 6, Training Loss: 1.171153789394102
Finish Epoch: 6, Validation Loss: 0.9684740534195533
Validation Accuracy: 0.7942708333333334
Finish Epoch: 7, Training Loss: 1.1534256230546942
Finish Epoch: 7, Validation Loss: 0.9560337632130353
Validation Accuracy: 0.80408653846153

Accuracy on validation,▁▃▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▆▇██▇█▇█▇▇████
Batch_size,▁
Epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
LR,▁
Training Loss,█▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,█▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▄▂▁▁▁▁▂▁▂▂▁▁▁▁
Accuracy on validation,0.85036
Batch_size,64
Epoch,32
LR,0.003
Training Loss,0.98184


In [93]:
resnet18 = resnet18.to(device)
resnet18.eval()
predictions = []  # List to store predicted labels

with torch.no_grad():
    for inputs, labels in test_iter:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet18(inputs)
        predicted = outputs.argmax(dim=1)
        predictions.extend(predicted.type(torch.int32).cpu().numpy())

# Create submission file
identifiers = list(range(1, len(test_ds) + 1))
identifiers.sort(key=lambda x: str(x))
submission_df = pd.DataFrame({'id': identifiers, 'label': predictions})
submission_df['label'] = submission_df['label'].apply(lambda x: train_ds.classes[x])
submission_df.to_csv('submission.csv', index=False)
